In [1]:
from jina import Client

In [2]:
from docarray import Document, DocumentArray
from multiprocessing import Process, Queue
import time
import uuid

In [3]:
client = Client(port=51005)

In [4]:
client.dry_run()

True

In [5]:
def get_doc():
    doc = Document(uri='https://raw.githubusercontent.com/fastai/fastai/master/nbs/images/puppy.jpg')
    return doc.load_uri_to_blob()

In [6]:
da = DocumentArray([get_doc() for _ in range(10)])

In [7]:
%%time
DocumentArray.from_protobuf(da.to_protobuf())

CPU times: user 1.81 ms, sys: 1.35 ms, total: 3.16 ms
Wall time: 2.59 ms


╭────────────────────── Documents Summary ──────────────────────╮
│                                                               │
│   Length                 10                                   │
│   Homogenous Documents   True                                 │
│   Common Attributes      ('id', 'blob', 'mime_type', 'uri')   │
│   Multimodal dataclass   False                                │
│                                                               │
╰───────────────────────────────────────────────────────────────╯
╭───────────────────── Attributes Summary ──────────────────────╮
│                                                               │
│   Attribute   Data type    #Unique values   Has empty value   │
│  ───────────────────────────────────────────────────────────  │
│   blob        ('bytes',)   1                False             │
│   id          ('str',)     10               False             │
│   mime_type   ('str',)     1                False             │
│   uri         ('str',)     1                False             │
│                                                               │
╰───────────────────────────────────────────────────────────────╯

In [8]:
%%time
client.index(inputs=[get_doc() for _ in range(10)])

CPU times: user 69.8 ms, sys: 37 ms, total: 107 ms
Wall time: 10.6 s


╭────────────────────── Documents Summary ──────────────────────╮
│                                                               │
│   Length                 10                                   │
│   Homogenous Documents   True                                 │
│   Common Attributes      ('id', 'blob', 'mime_type', 'uri')   │
│   Multimodal dataclass   False                                │
│                                                               │
╰───────────────────────────────────────────────────────────────╯
╭───────────────────── Attributes Summary ──────────────────────╮
│                                                               │
│   Attribute   Data type    #Unique values   Has empty value   │
│  ───────────────────────────────────────────────────────────  │
│   blob        ('bytes',)   1                False             │
│   id          ('str',)     10               False             │
│   mime_type   ('str',)     1                False             │
│   uri         ('str',)     1                False             │
│                                                               │
╰───────────────────────────────────────────────────────────────╯

In [9]:
da = DocumentArray([get_doc() for _ in range(10)])

In [10]:
def post_request(queue):
    """Full dalle flow run"""
    id_ = str(uuid.uuid1())
    init_time= time.time()
    
    print(f'hello {id_} \n')
    
    client = Client(port=51005)    
    client.index(inputs=da)    
    
    print(f'byebye {id_} \n time spent: {time.time()-init_time} \n queue size: {q.qsize()}')
    print(time.time()-init_time)
    q.put(id_)

In [15]:
N = 18

q = Queue()

list_thread = [Process(target=post_request, args=(q,), daemon=True) for _ in range(N)]

# + tags=[]
# %%time
for t in list_thread:
    t.start()
for t in list_thread:
    t.join()
# -

print(f'{N - q.qsize() } failed')

hello 3360fc18-12fb-11ed-8565-63455291752f 

hello 336157d0-12fb-11ed-8565-63455291752f 

hello 3361d75a-12fb-11ed-8565-63455291752f 

hello 3362a0ea-12fb-11ed-8565-63455291752f 

hello 336318ea-12fb-11ed-8565-63455291752f 
hello 3363ca6a-12fb-11ed-8565-63455291752f 

hello 33649a3a-12fb-11ed-8565-63455291752f 
hello 33650d76-12fb-11ed-8565-63455291752f 


hello 33659f52-12fb-11ed-8565-63455291752f 

hello 33664948-12fb-11ed-8565-63455291752f 

hello 3366d156-12fb-11ed-8565-63455291752f 

hello 336775a2-12fb-11ed-8565-63455291752f 

hello 33681ae8-12fb-11ed-8565-63455291752f 


hello 3368b0de-12fb-11ed-8565-63455291752f 

hello 33695e80-12fb-11ed-8565-63455291752f 
hello 336a0aec-12fb-11ed-8565-63455291752f 


hello 336a9642-12fb-11ed-8565-63455291752f 

hello 336b19fa-12fb-11ed-8565-63455291752f 



byebye 3360fc18-12fb-11ed-8565-63455291752f 
 time spent: 21.05617046356201 
 queue size: 0
21.056951999664307


byebye 336157d0-12fb-11ed-8565-63455291752f 
 time spent: 41.97665977478027 
 queue size: 1
41.97988176345825


ERROR  GRPCClient@64114 gRPC error: StatusCode.UNAVAILABLE Socket closed  [08/03/22 09:15:39]
       The ongoing request is terminated as the server is not available                      
       or closed already.                                                                    

Process Process-18:
Traceback (most recent call last):
  File "/home/sami/.pyenv/versions/3.9.10/lib/python3.9/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/sami/.pyenv/versions/3.9.10/lib/python3.9/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_62644/1901514774.py", line 9, in post_request
    client.index(inputs=da)
  File "/home/sami/.pyenv/versions/3.9.10/lib/python3.9/site-packages/jina/clients/mixin.py", line 185, in post
    return run_async(
  File "/home/sami/.pyenv/versions/3.9.10/lib/python3.9/site-packages/jina/helper.py", line 1323, in run_async
    return asyncio.run(func(*args, **kwargs))
  File "/home/sami/.pyenv/versions/3.9.10/lib/python3.9/asyncio/runners.py", line 44, in run
    return loop.run_until_complete(main)
  File "/home/sami/.pyenv/versions/3.9.10/lib/python3.9/asyncio/base_events.py", line 642, in run_until_complete
    return future.result()
  File "/ho

...

byebye 336775a2-12fb-11ed-8565-63455291752f 
 time spent: 1002.2208435535431 
 queue size: 12
1002.2216777801514


byebye 33695e80-12fb-11ed-8565-63455291752f 
 time spent: 1005.9855620861053 
 queue size: 13
1005.9864490032196


byebye 336a0aec-12fb-11ed-8565-63455291752f 
 time spent: 1009.0620384216309 
 queue size: 14
1009.062956571579


byebye 336a9642-12fb-11ed-8565-63455291752f 
 time spent: 1067.3257529735565 
 queue size: 15
1067.3265850543976


byebye 336b19fa-12fb-11ed-8565-63455291752f 
 time spent: 1068.3288254737854 
 queue size: 16
1068.3298342227936
1 failed


In [12]:
import numpy as np

In [13]:
matrix = np.random.rand(20_000, 4_000)

In [14]:
matrix.nbytes/ 1024**2

610.3515625